In [1]:
import pandas as pd
import plotly.express as px
import json
import dash
import dash_core_components as dcc
import dash_html_components as html
from jupyter_dash import JupyterDash
from dash.dependencies import Input, Output, State

In [2]:
# importing data
df = pd.read_csv('Motors.csv')

# Removing Nas from the data
df = df.dropna()

# CLeaning and Wrangling the data
df['RATING'] = df['RATING'].astype(float)
df['NUMBER'] = df['NUMBER'].astype(str).str.strip()
df['STARTER'] = df['STARTER'].astype(str).str.strip()

# Creating a new column called Function
df['FUNCTION'] = df['NUMBER'].apply(lambda x: 'PUMP' if x[3:5] == 'PP' else('AGITATOR' if  x[3:5] == 'AG' else (
    'BLOWERS' if x[3:5] == 'CP' else ('EQUIPED SPARE' if x[3:5] == 'EQ' else ('FEEDER' if x[3:5] == 'FE' else (
        'HEATER' if x[3:5] == 'HX' else ('SCREEN' if x[3:5] == 'SC' else ('THICKENER' if x[3:5] == 'TH' else (
            'MILL' if (x[3:5] == 'ML')|(x[3:6] == 'SAG')|(x[3:6] == 'BAL') else ('RBC' if x[3:5] == 'RB' else (
                'COOLING TOWER' if x[3:5] == 'C2' else ('FAN' if x[3:5] == 'FA' else 'OTHER'))))))))))))

# Creating a new column called Area
df['AREA'] = df['NUMBER'].apply(lambda x: 'CRUSHER/MILLING' if (x[:2] == '11')|(x[:2] == '12') else(
    'BIOX' if (x[:2] == '23') |(x[:2] == '13') else ('WTP' if x[:2] == '27' else ('TSF' if x[:2] == '17' else(
        'CIL' if x[:2] == '19' else 'REAGENTS' if x[:2] == '16' else (
            'ELUTION/GOLDROOM' if (x[:2] == '05')|(x[:2] == '15')|(x[:2] == '20') else (
                'FLOATATION' if (x[:2] == '18')|(x[:2] == '8') else 'BACKFILL' )) )))))

In [5]:
df.to_csv('cleaned.csv')

In [3]:
df = pd.read_csv('cleaned.csv')
# Sorting values
df = df.sort_values('AREA')


colors = {
    'background': '#659EC7',
    'text': '#111111'}
barcolor = '#f0bf60'

# Area against count
area_count = px.histogram(df, x = 'AREA', title = 'AREAS AGAINST THE SUM OF MOTORS', 
                          color_discrete_sequence=[barcolor])
area_count.update_layout(clickmode='event+select')
area_count.update_layout(
    plot_bgcolor=colors['background'],
    paper_bgcolor=colors['background'],
    font_color=colors['text']
)

# Area against ratings
loc_rat = df.groupby(['AREA'])['RATING'].sum().reset_index()
area_rating = px.bar(loc_rat, x = 'AREA', y = 'RATING', title = 'AREAS AGAINST SUM OF MOTOR RATINGS', 
                     color_discrete_sequence=[barcolor])
area_rating.update_layout(
    plot_bgcolor=colors['background'],
    paper_bgcolor=colors['background'],
    font_color=colors['text']
)


# MOTOR STARTERS
start_count = px.histogram(df.sort_values('STARTER'), x = 'STARTER', title = 'STARTERS AGAINST NUMBER OF MOTORS ',
                           color_discrete_sequence= [barcolor])
start_count.update_layout(
    plot_bgcolor=colors['background'],
    paper_bgcolor=colors['background'],
    font_color=colors['text']
)

# STARTERS AGAINST RATING
start_rat = df.groupby(['STARTER'])['RATING'].sum().reset_index()
star_rat = px.bar(start_rat, x = 'STARTER', y = 'RATING', title = 'STARTERS AGAINST SUM OF MOTOR RATINGS',
                  color_discrete_sequence=[barcolor])
star_rat.update_layout(
    plot_bgcolor=colors['background'],
    paper_bgcolor=colors['background'],
    font_color=colors['text']
)

# RATINGS RANGE
ranges = [1, 30, 50, 100, 200, 3800 ]
ran = df['RATING'].value_counts(bins=ranges, sort=False).reset_index()
ran = ran.rename(columns={"index": "Ratings Range", "RATING": "Number"})
pd.DataFrame(ran)
rang = ['1-30', '30-50', '50-100', '100-200', '200-3800']
ran['Range'] = rang
rat_count = px.bar(ran, x='Range', y='Number', 
                   title = 'RANGE OF RATINGS AGAINST NUMBER OF MOTORS',
                   color_discrete_sequence=[barcolor])
rat_count.update_layout(
    plot_bgcolor=colors['background'],
    paper_bgcolor=colors['background'],
    font_color=colors['text']
)


# Function against count
fun_count = px.histogram(df.sort_values('FUNCTION'), x = 'FUNCTION', 
                         title = 'FUNCTIONS AGAINST NUMBER OF MOTORS',
                         color_discrete_sequence=[barcolor])
fun_count.update_layout(
    plot_bgcolor=colors['background'],
    paper_bgcolor=colors['background'],
    font_color=colors['text']
)
    
# Function against rating
fun_rat = df.groupby(['FUNCTION'])['RATING'].sum().reset_index()
func_rat = px.bar(fun_rat, x = 'FUNCTION', y = 'RATING', 
                  title = 'FUNCTIONS AGAINST THE TOTAL NUMBER OF RATINGS', 
                  color_discrete_sequence=[barcolor])
func_rat.update_layout(
    plot_bgcolor=colors['background'],
    paper_bgcolor=colors['background'],
    font_color=colors['text']
) 


app = JupyterDash(__name__)

locations = df['AREA'].unique()

app.layout = html.Div(style ={'backgroundColor': '#848b79'}, children = [
#     html.H1(children="MOTOR DASHBOARD", 
#              style={
#                  'text-align':'center',
#                  'background': '#111111'}),


    html.H1(children = "GENERAL PLANT DASHBOARD",
            style={"text-align":"center"}),
    
    dcc.Graph(id="area_c", figure = area_count,
            style= {'width': '49%', 'display': 'inline-block',}),
    
    dcc.Graph(id="area_r", figure = area_rating,
            style= {'width': '49%', 'display': 'inline-block'}),
    
    dcc.Graph(id="start_c", figure = start_count,
            style= {'width': '29.5%', 'display': 'inline-block'}),


    dcc.Graph(id="start_r", figure= star_rat,
            style= {'width': '29.5%', 'display': 'inline-block'}),
    
    dcc.Graph(id="ratings", figure= rat_count,
            style= {'width': '39%', 'display': 'inline-block'}),
    
    dcc.Graph(id="function_c", figure= fun_count,
            style= {'width': '49%', 'display': 'inline-block'}),
    
    dcc.Graph(id="function_r", figure= func_rat,
            style= {'width': '49%', 'display': 'inline-block'}),
    
])

   
@app.callback([Output('start_c', 'figure'),
               Output('start_r', 'figure'),
               Output('ratings', 'figure'),
               Output('function_c', 'figure'),
               Output('function_r', 'figure')],
               Input('area_c', 'clickData'))

def update_graph(clickData):
    if clickData is None:
        return start_count, star_rat, rat_count, fun_count, func_rat
    else:
        dff = df.copy()
        dff = dff[dff['AREA']== clickData['points'][0]['x']]

        
        # MOTOR STARTERS
        start_count1 = px.histogram(dff.sort_values('STARTER'), x = 'STARTER', 
                                    title = 'STARTERS AGAINST NUMBER OF MOTORS ',
                                    color_discrete_sequence=[barcolor])
        start_count1.update_layout(
            plot_bgcolor=colors['background'],
            paper_bgcolor=colors['background'],
            font_color=colors['text']
) 
        
        # STARTERS AGAINST RATING
        start_rat1 = dff.groupby(['STARTER'])['RATING'].sum().reset_index()
        star_rat1 = px.bar(start_rat1, x = 'STARTER', y = 'RATING',
                           title = 'STARTERS AGAINST SUM OF MOTOR RATINGS', 
                           color_discrete_sequence=[barcolor])
        star_rat1.update_layout(
            plot_bgcolor=colors['background'],
            paper_bgcolor=colors['background'],
            font_color=colors['text']
) 
        


        # RATINGS RANGE
        rang = dff['RATING'].value_counts(bins=ranges, sort=False).reset_index()
        rang = rang.rename(columns={"index": "Ratings Range", "RATING": "Number"})
        pd.DataFrame(rang)
        rangg = ['1-30', '30-50', '50-100', '100-200', '200-3800']
        rang['Range'] = rangg
        rat_count1 = px.bar(rang, x='Range', y='Number', 
                            title = 'RANGE OF RATINGS AGAINST NUMBER OF MOTORS', 
                            color_discrete_sequence=[barcolor])
        rat_count1.update_layout(
            plot_bgcolor=colors['background'],
            paper_bgcolor=colors['background'],
            font_color=colors['text']
) 
        

          
        
       # Function against count
        fun_count1 = px.histogram(dff.sort_values('FUNCTION'), x = 'FUNCTION', 
                                  title = 'FUNCTIONS AGAINST NUMBER OF MOTORS', 
                                  color_discrete_sequence=[barcolor])
        fun_count1.update_layout(
            plot_bgcolor=colors['background'],
            paper_bgcolor=colors['background'],
            font_color=colors['text']
) 
        
        
        # Function against rating
        fun_rat1 = dff.groupby(['FUNCTION'])['RATING'].sum().reset_index()
        func_rat1 = px.bar(fun_rat1, x = 'FUNCTION', y = 'RATING',
                           title = 'FUNCTIONS AGAINST THE TOTAL NUMBER OF RATINGS', 
                           color_discrete_sequence=[barcolor])
        func_rat1.update_layout(
            plot_bgcolor=colors['background'],
            paper_bgcolor=colors['background'],
            font_color=colors['text']
) 
        



        return start_count1, star_rat1, rat_count1, fun_count1, func_rat1
              
if __name__ == '__main__':
    app.run_server(debug=True, port=8052)


Dash app running on http://127.0.0.1:8052/


,Unnamed: 0,NUMBER,NAME,RATING,STARTER,FUNCTION,AREA
0,158,23-PP-353,PLANT RUNOFF PUMP,75.0,DOL,PUMP,BIOX
113,618,23-PP-021A,NEUTRALIZATION TAILS PUMP 2,315.0,VSD,PUMP,BIOX
114,606,13-PP-003,PREGNANT ELUATE PUMP NO. 1,5.5,DOL,PUMP,BIOX
115,604,13-PP-022,FLOAT THICKNER TAILS PUMP 2,220.0,VSD,PUMP,BIOX
116,6,23-PP-028,SUMP PUMP,15.0,DOL,PUMP,BIOX
...,...,...,...,...,...,...,...
566,188,27-PP-813,RBC RECIRCULATION PUMP 2,7.5,DOL,PUMP,WTP
567,186,27-PP-816,RBC AREA SUMP PUMP,22.0,DOL,PUMP,WTP
568,187,27-PP-814,RBC NUTRIENTS FEED 1,7.5,DOL,PUMP,WTP
570,159,27-PP-842,RBC FEED PUMP 2,37.0,DOL,PUMP,WTP
